# Flatiron Health mPC: Final GBM build 

**OBJECTIVE: Build a gradient boosted survival model on the entirety of the Flation Health metastatic prostate cancer data set.**

**BACKGROUND: For details on hyperparameter tuning of the gradient boosted model see notebook *Machine learning crude imputation*. Missingness will be crudely imputatated given similar test-set AUC performance when compared to MICE.** 

**OUTLINE:**
1. **Preprocessing**
2. **Gradient boosted model** 
3. **Calculate patient risk scores**
4. **Export final list of variables**

## 1. Preprocessing 

In [1]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer 

In [2]:
# Function that returns number of rows and count of unique PatientIDs for a dataframe. 
def row_ID(dataframe):
    row = dataframe.shape[0]
    ID = dataframe['PatientID'].nunique()
    return row, ID

### Importing full dataframes

In [3]:
# Import training set and set PatientID as index.
train = pd.read_csv('train_full.csv')

In [4]:
row_ID(train)

(15141, 15141)

In [5]:
# Import test set and set PatientID as index.
test = pd.read_csv('test_full.csv')

In [6]:
row_ID(test)

(3786, 3786)

In [7]:
df = pd.concat([train, test])

In [8]:
row_ID(df)

(18927, 18927)

In [9]:
df = df.set_index('PatientID')

In [10]:
df.sample(3)

,Gender,race,ethnicity,age,p_type,NStage,MStage,Histology,GleasonScore,PSADiagnosis,...,peritoneum_met,liver_met,other_gi_met,cns_met,bone_met,lymph_met,kidney_bladder_met,other_met,prim_treatment,early_adt
PatientID,,,,,,,,,,,,,,,,,,,,,
F460A5F25DC6F,M,white,unknown,66,COMMUNITY,N0,M1b,Adenocarcinoma,9,32.8,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,unknown,0.0
F709BC26937A1,M,white,unknown,67,COMMUNITY,N0,M1,Adenocarcinoma,3 + 4 = 7,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unknown,0.0
FC703172535FB,M,white,unknown,78,COMMUNITY,Unknown / Not documented,Unknown / Not documented,Adenocarcinoma,Unknown / Not documented,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,radiation,1.0


### Converting datatypes 

In [11]:
list(df.select_dtypes(include = ['object']).columns)

['Gender',
 'race',
 'ethnicity',
 'p_type',
 'NStage',
 'MStage',
 'Histology',
 'GleasonScore',
 'stage',
 'brca_status',
 'ecog_diagnosis',
 'prim_treatment']

In [12]:
to_be_categorical = list(df.select_dtypes(include = ['object']).columns)

In [13]:
to_be_categorical.append('met_year')

In [14]:
# Convert variables in list to categorical.
for x in list(to_be_categorical):
    df[x] = df[x].astype('category')

In [15]:
list(df.select_dtypes(include = ['category']).columns)

['Gender',
 'race',
 'ethnicity',
 'p_type',
 'NStage',
 'MStage',
 'Histology',
 'GleasonScore',
 'stage',
 'met_year',
 'brca_status',
 'ecog_diagnosis',
 'prim_treatment']

In [16]:
# Convert death_status into True or False (required for scikit-survival). 
df['death_status'] = df['death_status'].astype('bool')

### Dropping unneeded labs 

In [17]:
drop_labs = [
    'albumin_avg',
    'alp_avg',
    'alt_avg',
    'ast_avg',
    'bicarb_avg',
    'bun_avg',
    'calcium_avg',
    'chloride_avg',
    'creatinine_avg',
    'hemoglobin_avg',
    'neutrophil_count_avg',
    'platelet_avg',
    'potassium_avg',
    'psa_avg',
    'sodium_avg',
    'total_bilirubin_avg',
    'wbc_avg',
    'albumin_max',
    'bicarb_max',
    'bun_max',
    'chloride_max',
    'hemoglobin_max',
    'neutrophil_count_max',
    'platelet_max',
    'potassium_max',
    'sodium_max',
    'alp_min',
    'alt_min',
    'ast_min',
    'bun_min',
    'calcium_min',
    'chloride_min',
    'creatinine_min',
    'neutrophil_count_min',
    'potassium_min',
    'psa_min',
    'total_bilirubin_min',
    'albumin_std',
    'alp_std',
    'alt_std',
    'ast_std',
    'bicarb_std',
    'bun_std',
    'calcium_std',
    'chloride_std',
    'creatinine_std',
    'hemoglobin_std',
    'neutrophil_count_std',
    'platelet_std',
    'potassium_std',
    'psa_std',
    'sodium_std',
    'total_bilirubin_std',
    'wbc_std',
    'albumin_slope',
    'alp_slope',
    'alt_slope',
    'ast_slope',
    'bicarb_slope',
    'bun_slope',
    'calcium_slope',
    'chloride_slope',
    'creatinine_slope',
    'hemoglobin_slope',
    'neutrophil_count_slope',
    'platelet_slope',
    'potassium_slope',
    'sodium_slope',
    'total_bilirubin_slope',
    'wbc_slope',
    'albumin_slope_na',
    'alp_slope_na',
    'alt_slope_na',
    'ast_slope_na',
    'bicarb_slope_na',
    'bun_slope_na',
    'calcium_slope_na',
    'chloride_slope_na',
    'creatinine_slope_na',
    'hemoglobin_slope_na',
    'neutrophil_count_slope_na',
    'platelet_slope_na',
    'potassium_slope_na',
    'sodium_slope_na',
    'total_bilirubin_slope_na',
    'wbc_slope_na']

In [18]:
df.shape

(18927, 227)

In [19]:
df = df.drop(columns = drop_labs)

In [20]:
df.shape

(18927, 141)

In [21]:
df.loc[:, 'alp_max_na'] = np.where(df['alp_max'].isna(), 1, 0)
df.loc[:, 'alt_max_na'] = np.where(df['alt_max'].isna(), 1, 0)
df.loc[:, 'ast_max_na'] = np.where(df['ast_max'].isna(), 1, 0)
df.loc[:, 'calcium_max_na'] = np.where(df['calcium_max'].isna(), 1, 0)
df.loc[:, 'creatinine_max_na'] = np.where(df['creatinine_max'].isna(), 1, 0)
df.loc[:, 'psa_max_na'] = np.where(df['psa_max'].isna(), 1, 0)
df.loc[:, 'total_bilirubin_max_na'] = np.where(df['total_bilirubin_max'].isna(), 1, 0)
df.loc[:, 'wbc_max_na'] = np.where(df['wbc_max'].isna(), 1, 0)
df.loc[:, 'albumin_min_na'] = np.where(df['albumin_min'].isna(), 1, 0)
df.loc[:, 'bicarb_min_na'] = np.where(df['bicarb_min'].isna(), 1, 0)
df.loc[:, 'hemoglobin_min_na'] = np.where(df['hemoglobin_min'].isna(), 1, 0)
df.loc[:, 'platelet_min_na'] = np.where(df['platelet_min'].isna(), 1, 0)
df.loc[:, 'sodium_min_na'] = np.where(df['sodium_min'].isna(), 1, 0)
df.loc[:, 'wbc_min_na'] = np.where(df['wbc_min'].isna(), 1, 0)

In [22]:
df.shape

(18927, 155)

### Separate into X and Y 

In [23]:
# 'X' datasets
df_x = df.drop(columns = ['death_status', 'timerisk_activity']) #80% of data 

In [24]:
# 'Y' datasets
# Death status and time until event needs to be stored as a structured array to be compatible with scikit-survival
y_dtypes = df[['death_status', 'timerisk_activity']].dtypes

df_y = np.array([tuple(x) for x in df[['death_status', 'timerisk_activity']].values],
                dtype = list(zip(y_dtypes.index, y_dtypes)))

### Pipeline 

In [25]:
# List of numeric variables, excluding binary variables. 
numerical_features = [
    'age',
    'PSADiagnosis',
    'PSAMetDiagnosis',
    'delta_met_diagnosis',
    'crpc_time',
    'weight_diag',
    'bmi_diag',
    'weight_pct_change',
    'weight_slope',
    'albumin_diag',
    'alp_diag',
    'alt_diag',
    'ast_diag',
    'bicarb_diag',
    'bun_diag',
    'calcium_diag',
    'chloride_diag',
    'creatinine_diag',
    'hemoglobin_diag',
    'neutrophil_count_diag',
    'platelet_diag',
    'potassium_diag',
    'sodium_diag',
    'total_bilirubin_diag',
    'wbc_diag',
    'alp_max',
    'alt_max',
    'ast_max',
    'calcium_max',
    'creatinine_max',
    'psa_max',
    'total_bilirubin_max',
    'wbc_max',
    'albumin_min',
    'bicarb_min',
    'hemoglobin_min',
    'platelet_min',
    'sodium_min',
    'wbc_min',
    'psa_slope',
    'icd_count']

# Transformer will impute column medians and then apply a standard scaler. 
numerical_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'median')),
    ('std_scaler', StandardScaler())])

In [26]:
# List of categorical features.
categorical_features = list(df_x.select_dtypes(include = ['category']).columns)

# One-hot-encode categorical features.
categorical_transformer = OneHotEncoder(handle_unknown = 'ignore')

In [27]:
preprocessor = ColumnTransformer(
    transformers = [
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)],
    remainder = 'passthrough')

## 2. Gradient boosted model 

In [28]:
from sklearn.pipeline import make_pipeline

from sksurv.ensemble import GradientBoostingSurvivalAnalysis

from joblib import dump, load 

In [29]:
df_x.shape

(18927, 153)

In [30]:
df_y.shape

(18927,)

In [31]:
gbm_final = make_pipeline(preprocessor, GradientBoostingSurvivalAnalysis(n_estimators = 675,
                                                                         learning_rate = 0.05,
                                                                         max_depth = 4,
                                                                         subsample = 0.50,
                                                                         verbose = 1,
                                                                         random_state = 42))

gbm_final.fit(df_x, df_y)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1       40495.0727          28.5857           51.78m
         2       40457.1387          28.1595           51.30m
         3       40381.6984          26.4578           51.07m
         4       40668.6698          25.7462           50.90m
         5       40055.9420          24.1777           50.78m
         6       40593.2834          24.0648           50.64m
         7       40681.8998          23.0587           50.59m
         8       40245.9290          21.2085           50.46m
         9       40339.8004          21.1435           50.33m
        10       40050.7591          20.2727           50.26m
        20       40244.8276          14.3636           49.43m
        30       40448.6344           9.6662           48.65m
        40       39774.3243           8.7681           47.87m
        50       40234.2404           5.9704           47.10m
        60       39699.6484           5.2839           46.32m
       

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('std_scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'PSADiagnosis',
                                                   'PSAMetDiagnosis',
                                                   'delta_met_diagnosis',
                                                   'crpc_time', 'weight_diag',
                                                   'bmi_diag',
                                                   'weight_pct_change',
                                                   'weight_slope

In [32]:
dump(gbm_final, 'gbm_final.joblib') 

['gbm_final.joblib']

## 3. Calculate patient risk scores 

In [33]:
# Calculate risk score for patients in training set. 
crude_risk_score_df = pd.DataFrame({'risk_score': gbm_final.predict(df_x)},
                                   index = df_x.index)

In [34]:
crude_risk_score_df.to_csv('crude_risk_score_df.csv', index = True, header = True)

## 4. Export final list of variables

In [35]:
import csv

In [36]:
prostate_columns = list(df_x.columns)

In [37]:
with open('prostate_columns.csv', 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(prostate_columns)